In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

data = {}
for i in range(470, 476):
    data[i] = pd.read_csv('/Users/ymdt/src/dreem_nap/ex/Lauren/{}.csv'.format(i))
df = pd.concat(data).reset_index().drop(columns=['Unnamed: 0','DMS_conc','DMS_conc_type','level_1']).rename(columns={'level_0':'sample'})


df.reset_index(inplace=True, drop=True)
for col in [ 'mut_bases', 'info_bases','del_bases','ins_bases','cov_bases'] + \
    [c for c in df.columns.tolist() if (c.startswith('mod_bases'))]:
    df[col] = df[col].apply(lambda x: [float(b) for b in x[1:-1].replace('\n',' ').replace(',',' ').split(' ') if b != ''][1:])

df['mut_rates'] = df.apply(lambda x: list(np.round(np.array(x['mut_bases'])/np.array(x['info_bases']), 8)), axis=1)

if not 'worst_cov_bases' in df.columns:
    df['worst_cov_bases'] = df['cov_bases'].apply(lambda x: min(x))

for attr in ['section','cluster']:
    if attr not in df.columns:
        df[attr] = 0

sections = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    def lah(row,col): 
        return row[col][row['sequence'].index('CACAGTCGAAAGACTGTG'):row['sequence'].index('CACAGTCGAAAGACTGTG')+len('CACAGTCGAAAGACTGTG')]
    def ms2(row,col): 
        return row[col][19:42]
    def roi(row,col): 
        return row[col][45:row['sequence'].index('CACAGTCGAAAGACTGTG')-2]
    for fun in [lah, ms2, roi]:
        add_row = row.copy()
        add_row = add_row[['sample','construct','cluster']] 
        add_row['section'] = fun.__name__
        for col in [ 'mut_bases', 'info_bases','del_bases','ins_bases','cov_bases','mut_rates','sequence','structure'] + \
            [c for c in df.columns.tolist() if (c.startswith('mod_bases') or c.startswith('poisson'))]:
            if col not in ['sequence','structure'] and type(row[col]) == str:
                row[col] = [float(b) for b in row[col][1:-1].replace('\n',' ').replace(',',' ').split(' ') if b != '']
            add_row[col] = fun(row,col)
        add_row['worst_cov_bases'] = min(add_row['cov_bases'])
        sections.append(add_row)
df = pd.concat((df,pd.concat(sections, axis=1).T.reset_index().drop(columns=['index'])))

df['family'] = df['construct'].apply(lambda x: x[x.index('=')+1:x.index('=')+x[x.index('='):].index('-')]) # TODO
df.to_csv('/Users/ymdt/src/dreem_nap/ex/Lauren/lauren_2.csv', index=False)
df = df.iloc[np.random.choice(len(df), 100, replace=False)]
df.to_csv('/Users/ymdt/src/dreem_nap/ex/Lauren/lauren_small_2.csv', index=False)


/var/folders/00/r1kkxj7d0k344frrqtzrc5000000gn/T/ipykernel_73552/2435524770.py:16: RuntimeWarning: invalid value encountered in divide
  df['mut_rates'] = df.apply(lambda x: list(np.round(np.array(x['mut_bases'])/np.array(x['info_bases']), 8)), axis=1)
100%|██████████| 3767/3767 [00:24<00:00, 152.17it/s]


In [3]:
pd.concat(sections, axis=1).T.reset_index().drop(columns=['index'])

,sample,construct,cluster,section,mut_bases,info_bases,del_bases,ins_bases,cov_bases,mut_rates,...,structure,poisson_low,poisson_high,poisson_min,poisson_max,mod_bases_A,mod_bases_C,mod_bases_G,mod_bases_T,worst_cov_bases
0,470,3114-O-flank_1=hp7-DB,0,lah,"[36.0, 26.0, 6.0, 23.0, 3.0, 0.0, 9.0, 6.0, 15...","[2016.0, 2016.0, 2017.0, 2017.0, 2017.0, 2017....","[0.0, 8.0, 0.0, 7.0, 0.0, 2.0, 1.0, 3.0, 4.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1968.0, 1970.0, 1972.0, 1958.0, 1963.0, 1968....","[0.01785714, 0.01289683, 0.00297471, 0.0114030...",...,(((((((....))))))),"[0.00535022, 0.00447219, 0.00188305, 0.0041745...","[0.00686467, 0.00600001, 0.00349999, 0.0057071...","[0.0125069233, 0.00842463453, 0.00109166795, 0...","[0.02472181, 0.01889684, 0.0064747, 0.01711021...","[12.0, 0.0, 4.0, 0.0, 0.0, 0.0, 2.0, 3.0, 0.0,...","[0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0,...","[6.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 101.0...","[18.0, 23.0, 2.0, 19.0, 3.0, 0.0, 7.0, 2.0, 41...",1909.0
1,470,3114-O-flank_1=hp7-DB,0,ms2,"[8.0, 12.0, 23.0, 3.0, 18.0, 1.0, 1.0, 101.0, ...","[2013.0, 2013.0, 2013.0, 2013.0, 2013.0, 2013....","[1.0, 0.0, 5.0, 1.0, 0.0, 0.0, 1.0, 4.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1936.0, 1990.0, 2001.0, 2004.0, 1999.0, 2000....","[0.00397417, 0.00596125, 0.01142573, 0.0014903...",...,)((((((.((....)))))))).,"[0.0022584, 0.00288099, 0.0041828, 0.00118297,...","[0.00385653, 0.00445186, 0.00571848, 0.0028650...","[0.00171576362, 0.00308026583, 0.00724293444, ...","[0.0078307, 0.01041311, 0.01714421, 0.00435533...","[4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, ...","[0.0, 0.0, 5.0, 2.0, 0.0, 0.0, 0.0, 25.0, 0.0,...","[0.0, 5.0, 0.0, 1.0, 2.0, 1.0, 0.0, 30.0, 0.0,...","[4.0, 5.0, 18.0, 0.0, 16.0, 0.0, 1.0, 46.0, 6....",1936.0
2,470,3114-O-flank_1=hp7-DB,0,roi,"[16.0, 7.0, 15.0, 14.0, 23.0, 9.0, 16.0, 18.0,...","[2013.0, 2013.0, 2013.0, 2014.0, 2014.0, 2014....","[2.0, 9.0, 3.0, 0.0, 19.0, 10.0, 1.0, 2.0, 3.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1955.0, 1900.0, 1897.0, 1863.0, 1873.0, 1950....","[0.00794834, 0.0034774, 0.00745156, 0.00695134...",...,((((((((....))))))))),"[0.00340518, 0.0020793, 0.00328098, 0.00315098...","[0.00495926, 0.00368737, 0.00483866, 0.0047118...","[0.00454316068, 0.00139809392, 0.00417058427, ...","[0.0129076, 0.00716477, 0.01229022, 0.01166317...","[6.0, 1.0, 0.0, 3.0, 5.0, 4.0, 5.0, 5.0, 8.0, ...","[0.0, 2.0, 3.0, 2.0, 8.0, 5.0, 0.0, 0.0, 0.0, ...","[2.0, 0.0, 4.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[8.0, 4.0, 8.0, 9.0, 10.0, 0.0, 10.0, 13.0, 10...",1863.0
3,470,3482-O-flank_1=lp11-DB,0,lah,"[21.0, 36.0, 26.0, 17.0, 19.0, 6.0, 142.0, 7.0...","[2930.0, 2930.0, 2931.0, 2931.0, 2932.0, 2932....","[1.0, 9.0, 7.0, 9.0, 93.0, 1.0, 1.0, 0.0, 3.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2871.0, 2880.0, 2891.0, 2874.0, 2855.0, 2863....","[0.00716724, 0.01228669, 0.00887069, 0.0058000...",...,(((((((....))))))),"[0.0027306, 0.00368124, 0.00307606, 0.00242131...","[0.00378865, 0.00472327, 0.00412693, 0.0034864...","[0.00443663174, 0.00860544622, 0.00579463092, ...","[0.01095588, 0.01700996, 0.01299762, 0.0092864...","[9.0, 0.0, 6.0, 0.0, 9.0, 3.0, 129.0, 3.0, 0.0...","[0.0, 0.0, 0.0, 2.0, 0.0, 3.0, 0.0, 3.0, 18.0,...","[6.0, 5.0, 8.0, 0.0, 0.0, 0.0, 2.0, 0.0, 167.0...","[6.0, 31.0, 12.0, 15.0, 10.0, 0.0, 11.0, 1.0, ...",2823.0
4,470,3482-O-flank_1=lp11-DB,0,ms2,"[10.0, 23.0, 50.0, 3.0, 80.0, 5.0, 11.0, 148.0...","[2929.0, 2929.0, 2929.0, 2929.0, 2929.0, 2929....","[5.0, 17.0, 513.0, 0.0, 5.0, 1.0, 0.0, 3.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2794.0, 2900.0, 2910.0, 2908.0, 2898.0, 2903....","[0.00341413, 0.00785251, 0.01707067, 0.0010242...",...,)((((((.((....)))))))).,"[0.00177692, 0.00287469, 0.00440049, 0.0008130...","[0.00286458, 0.00393011, 0.00543488, 0.0019690...","[0.00163721021, 0.00497781736, 0.0126701822, 0...","[0.00627871, 0.01178262, 0.02250555, 0.0029932...","[5.0, 10.0, 0.0, 